In [ ]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dropout, Flatten, Dense, Activation
from keras.optimizers import Adam
from keras import regularizers
import matplotlib.pyplot as plt

#import zipfile
#import os

from google.colab import drive
drive.mount('/content/drive')

# Шлях до архіву
#zip_path = '/content/drive/MyDrive/AI/Pekman/data_arch.zip'

# Директорія, куди розпакувати дані
#extract_path = '/content/drive/MyDrive/AI/Pekman/'

#with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    #zip_ref.extractall(extract_path)






Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_loc = '/content/drive/MyDrive/AI/Pekman/train'
test_loc = '/content/drive/MyDrive/AI/Pekman/test'

# Використовуємо ImageDataGenerator для автоматичного препроцесінгу зображень під час тренування
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Розділіть дані на тренування та валідацію
)

test_datagen = ImageDataGenerator(rescale=1./255)

img_size = 48

# Передайте train_datagen та test_datagen до flow_from_directory
train_data = train_datagen.flow_from_directory(
    directory=train_loc,
    target_size=(img_size, img_size),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical",
    subset="training"
)

test_data = test_datagen.flow_from_directory(
    directory=test_loc,
    target_size=(img_size, img_size),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(48, 48, 1)))
model.add(Activation('mish'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('mish'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (5, 5), padding='same'))
model.add(Activation('mish'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01)))
model.add(Activation('mish'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('mish'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(256))
model.add(Activation('mish'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

epochs = 60
batch_size = 64

model.summary()

history = model.fit(x=train_data, epochs=epochs, validation_data=test_data)

fig, ax = plt.subplots(1, 2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
fig.set_size_inches(12, 4)

# Plotting Training and Validation Accuracy

ax[0].plot(train_acc)
if 'val_accuracy' in history.history:
    ax[0].plot(history.history['val_accuracy'])
    ax[0].legend(['Train', 'Validation'], loc='upper left')
else:
    ax[0].legend(['Train'], loc='upper left')
ax[0].set_title('Training Accuracy vs Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')


# Plotting Training and Validation Loss
ax[1].plot(train_loss)
if 'val_loss' in history.history:
    ax[1].plot(history.history['val_loss'])
    ax[1].legend(['Train', 'Validation'], loc='upper left')
else:
    ax[1].legend(['Train'], loc='upper left')
ax[1].set_title('Training Loss vs Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')

plt.show()

model.save('/content/drive/MyDrive/AI/Pekman/my_data/emotions_class_model.h5')

